In [1]:
import numpy as np
from ultralytics import YOLO
import os
from pathlib import Path
import cv2

os.chdir("C:/Users/memes/Documents/SIDS_revelation_project")
model = YOLO(".\\2.weights\\best.pt")

classes_fd = model.names
print(f"Face detection model is able to detect following classes:\n{classes_fd}")

Face detection model is able to detect following classes:
{0: 'BABY', 1: 'EYE', 2: 'HEAD', 3: 'MOUTH', 4: 'NOSE'}


Extract form json file


In [ ]:
import json
os.chdir("C:\\Users\\memes\\Documents\\SIDS_revelation_project")

file_imgid = {}
imgid_label = {}
file_label = {}

classes_mlp = {}

with open("train\\_annotations.coco.json", "r") as f:
    dataset = json.load(f)
    
    for img in dataset["images"]:
        file_imgid[img["file_name"]] =img["id"]
    
    for label in dataset["annotations"]:
        imgid_label[label["image_id"]] = label["category_id"]
    
    for cls in dataset["categories"]:
        classes_mlp[cls["name"]]=cls["id"]
    del classes_mlp["baby"]

for file, img_id in file_imgid.items():
    if img_id in imgid_label.keys():
        file_label[file] = imgid_label[img_id]      

In [ ]:
from ultralytics.engine.results import Boxes  

def embedding_builder_mery(prediction: Boxes):
    '''
    DEBUG PURPOSE
    if prediction.xywhn.size()[0]>6:
        print(f"Warning, more than 6 features detected in the sample\n{prediction.cls}")
    '''
    
    embedding_position= {
        "BABY" : 0,
        "HEAD": 1,
        "EYE": (2,3),
        "MOUTH": 4,
        "NOSE": 5
    }
    
    vec = np.full(6, -1.0, dtype=float)
    for cls in (prediction.cls):
        class_label = classes_fd[cls.item()]
        if class_label == "EYE":
            pos = embedding_position[class_label][0] if vec[embedding_position[class_label][0]] == -1 else embedding_position[class_label][1]
            vec[pos] = 1
        else:
            pos = embedding_position[class_label]
            vec[pos] = 1 
    return vec

In [ ]:
from ultralytics.engine.results import Boxes  

def embedding_builder(prediction: Boxes):
    '''
    DEBUG PURPOSE
    if prediction.xywhn.size()[0]>6:
        print(f"Warning, more than 6 features detected in the sample\n{prediction.cls}")
    '''
    
    embedding_position= {
        "BABY" : 0,
        "HEAD": 1,
        "EYE": (3, 5),
        "MOUTH": 7,
        "NOSE": 9
    }
    
    vec = np.full(11, -1.0, dtype=float)
    for bbox, cls in zip(prediction.xywhn, prediction.cls):
        class_label = classes_fd[cls.item()]
        if class_label == "BABY":
            pos = embedding_position[class_label]
            
            vec[pos] = 1
        elif class_label == "EYE":
            pos = embedding_position[class_label][0] if vec[embedding_position[class_label][0]] == -1 else embedding_position[class_label][1]
            
            vec[pos] = bbox[0]
            vec[pos+1] = bbox[1]
        else:
            pos = embedding_position[class_label]
            
            vec[pos] = bbox[0]
            vec[pos+1] = bbox[1]    
    
    return vec

In [ ]:
os.chdir("C:\\Users\\memes\\Documents\\SIDS_revelation_project")
onback_onstomach_dataset = Path("train/")
X = []
y = []

for img in onback_onstomach_dataset.glob("*.jpg"):
    if img.name in file_label.keys():
        #DEBUG
        if len(y)%100 == 0:
            print(f"{int(len(y)*100/len(file_label))}%-->    {len(y)} / {len(file_label)} embeddings created")
        #DEBUG    
        result = model(img, conf=0.3, verbose=False)[0]
        prediction = result.boxes
        
        vec = embedding_builder(prediction)
        X.append(vec)
        y.append(file_label[img.name])
        
print(f"FINISH, {len(y)} embeddings created")

In [ ]:
 #salvo gli imbedding in un file csv 
import csv

with open("train/processed_dataset.csv", "w", newline='') as f:
    writer = csv.writer(f)
    for xi, yi in zip(X, y):
        writer.writerow(list(xi) + [yi])

Then in order to extract X and y from .csv and load them in numpy arrays

In [ ]:
import numpy as np

data = np.loadtxt("train/processed_dataset.csv", delimiter=",")
X = data[:, :-1]
y = data[:, -1]

In [ ]:
X.shape

RANDOM FORESTTT

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Convert to NumPy arrays (if not already)
X_array = np.array(X)
y_array = np.array(y)

# Split into train/test sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=42)

# Create the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))